## ADA501 - Prosjekt

In [2]:
# !pip install numpy_financial

In [3]:
import pandas as pd
import numpy as np
import sympy as sp
from collections import namedtuple
from scipy.integrate import quad
from scipy.optimize import newton
from scipy.integrate import solve_ivp

In [4]:
k1 = 0.1        # Økning i forbruk etter inntekt                    -> hurtig økning etter inntektsøkning
k2 = 0.7        # Reduksjon i forbruk etter inntekt                 -> sakte endring etter inntektsfall
k3 = 0.3        # Renteendring basert på forbruk                    -> Forbruk øker -> Renter øker  -> Lånet påvirkes -> Må bruke mindre penger
k4 = 0.025       # Endring i inflasjon basert på forbuk & rente      -> Endres 0,25 poeng oftest

In [5]:
I0 = 10000      # Startinntekt pr måend
U0 = 6000       # Faste utgifter
F0 = 5000       # Initiale forbruksvaner
R0 = 0.05       # Startboliglånsrenterente
S0 = 0          # Ønsker å starte med 0 kr i overskudd, men fast inntekt
S0 = 300000     # Startsluttresultat over 25 år, vil ha et overskudd på konto

In [6]:
t0 = 0
t1 = 25

# Definer tidsintervall og initialverdier
t_span = (t0, t1)
y0 = [I0, U0, F0, R0]

## les inn csv filer

In [23]:
df = pd.read_excel('Siste.xlsx')

,Dato,Hva,Brukt
0,2020-01-02,Luksus,"-147,1"
1,2020-01-02,Luksus,-268
2,2020-01-02,Luksus,-399
3,2020-01-03,Inntekt,128
4,2020-01-03,Luksus,-250
...,...,...,...
1096,2023-10-02,Faste utgifter,-121
1097,2023-10-10,Inntekt,105
1098,2023-10-12,Inntekt,"9402,74"
1099,2023-10-12,Luksus,"-6081,09"


In [25]:
# Filter 'Inntekt'
inntekt = df[df['Hva'] == 'Inntekt']
# Filter 'Faste utgifter'
faste_utgifter = df[df['Hva'] == 'Faste utgifter']
# Filter 'Luksus'
forbruk = df[df['Hva'] == 'Luksus']

## Kommen hit

In [8]:
def forbruksvaner(t, F):
    dFdt = k1 * (I - F) - k2 * dIdF
    return dFdt

In [9]:
def inflasjon(t, I):
    dIdt = k4 * (F - I)
    return dIdt

In [10]:
def compute_loan_terms(years):
    # Define the variables and parameters
    t = sp.symbols('t')             # Time in years
    A = sp.Function('A')(t)         # Income function as a function of time
    R = sp.Function('R')(t)         # Interest rate function as a function of time

    # Define the integral equation for dL/dR
    integrand = -t * A * sp.exp(-R * t)
    dLdR = sp.integrate(integrand, (t, 0, years))

    # Return the result
    return dLdR

# Example usage:
years = 5  # Change this to the number of years you want
result = compute_loan_terms(years)
print(result)

-Integral(t*A(t)*exp(-t*R(t)), (t, 0, 5))


In [11]:
def styringsrente(t, I, R):
    dRdt = k3 * I
    return dRdt

In [12]:
def economic_system(t, y):
    I, U, F, R, L = y
    dFdt = forbruksvaner(t, F)
    dIdt = inflasjon(t, I)
    dRdt = styringsrente(t, I, R)
    dLdt = f(t, L, r)  # Implementer denne funksjonen basert på dine data
    dUdt = ...  # Implementer hvordan faste utgifter endres over tid
    return [dIdt, dUdt, dFdt, dRdt, dLdt]

In [13]:
initial_conditions = [I0, F0]  # Initielle betingelser
sol = solve_ivp(Inntekt, t_span, initial_conditions, args=(k1, k2))

TypeError: Inntekt() takes 2 positional arguments but 4 were given